In [ ]:
using fermions, Plots, Measures, ProgressMeter
Plots.theme(:dark)

In [ ]:
function TightBindHamiltonian(numSites)
    hamiltonianTerms = Tuple{String, Vector{Int64}, Float64}[]
    for i in 1:numSites-1
        term1 = ("+-", [i, i+1], 1.0)
        term2 = ("+-", [i+1, i], 1.0)
        push!(hamiltonianTerms, term1)
        push!(hamiltonianTerms, term2)
    end
    return hamiltonianTerms
end

In [ ]:
numSites = 200
basis_N = fermions.BasisStates(numSites, 1)
eigvals_N, eigvecs_N = fermions.Spectrum(TightBindHamiltonian(numSites), basis_N);

In [ ]:
@time SEE = fetch.(@showprogress [Threads.@spawn fermions.vnEntropy(eigvecs_N[1], collect(i-4:i+4)) for i in 5:numSites-4])
Plots.plot(SEE, thickness_scaling=1.4, linewidth=3, legend=false,
        xlabel="subsystem size", ylabel="subsystem entanglement", margin=-1mm)

In [ ]:
invTempRange = (10 .^ range(-0.5, stop=3.1, length=20)) ./ (maximum(eigvals_N) - minimum(eigvals_N))
leftOccOperator = [("n", [1], 1.0)]
leftOccAverage = @showprogress [fermions.ThermalAverage(eigvecs_N, eigvals_N, leftOccOperator, beta) for beta in invTempRange]
centerOccOperator = [("n", [div(numSites, 2)], 1.0)]
centerOccAverage = @showprogress [fermions.ThermalAverage(eigvecs_N, eigvals_N, centerOccOperator, beta) for beta in invTempRange]
Plots.plot(1 ./ invTempRange, [leftOccAverage, centerOccAverage];
    thickness_scaling=1.5, linewidth=3, 
    xaxis=:log10, xlabel="temp. / \$\\Delta E\$", ylabel="local occupancy", 
    labels=["left edge" "center"], leftmargin=-5mm, bottommargin=-3mm)

In [ ]:
for numSites in [8, 20, 60]
    basis_N = fermions.BasisStates(numSites, 1)#; totOccCriteria=(x,N)->sum(x) == 1)
    basis_Nm1 = fermions.BasisStates(numSites, 0)#; totOccCriteria=(x,N)->sum(x) == 0)
    basis_Np1 = fermions.BasisStates(numSites, 2)#; totOccCriteria=(x,N)->sum(x) == 2)
    tightBindHam = TightBindHamiltonian(numSites)
    eigvals_N, eigvecs_N = fermions.Spectrum(tightBindHam, basis_N)
    eigvals_Nm1, eigvecs_Nm1 = fermions.Spectrum(tightBindHam, basis_Nm1)
    eigvals_Np1, eigvecs_Np1 = fermions.Spectrum(tightBindHam, basis_Np1)
    freqArr = collect(range(-2.2, stop=2.2, step=0.01))
    probe = [("-", [div(numSites, 2)], 1.0)]
    probeDag = [("+", [div(numSites, 2)], 1.0)]
    specfunc = fermions.SpecFunc((eigvals_N[1], eigvecs_N[1]), [eigvals_Nm1; eigvals_Np1], 
        [eigvecs_Nm1; eigvecs_Np1], probe, probeDag, freqArr, 1e-2)
    p = Plots.plot(freqArr, specfunc, thickness_scaling=1.5, linewidth=2, legend=false, 
        xlabel="frequency \$\\omega\$", ylabel="spectral func. \$A(\\omega)\$", margin=-2mm)
    display(p)
end